# EDA

# data split

원본 이미지를 train / test로 나누기

In [7]:
import os
import shutil

In [18]:
abnormal_lst = os.listdir('../data/abnormal')

print(len(abnormal_lst))
abnormal = []
for ab in abnormal_lst:
    abnormal.append(ab.split('.')[0])
    
    
print(len(abnormal))

44
44


In [19]:
abnormal[0]

'58581e6ee4b005193719746c'

In [12]:
data_lst = os.listdir('../src')

len(data_lst)

4644

In [20]:
txt_lst, jpg_lst = [], []
for data in data_lst:
    if data.split('.')[0] not in abnormal:
        if data.endswith('.txt'):
            txt_lst.append(data)
        else:
            jpg_lst.append(data)
    
txt_lst.sort()
jpg_lst.sort()

print(len(txt_lst), len(jpg_lst))

2278 2278


In [22]:
# 9:1 비율(count 계산)
int(len(txt_lst)*0.9), int(len(txt_lst)*0.1)

(2050, 227)

In [23]:
if not(os.path.isdir('../data/')):
        os.makedirs(os.path.join('../data/'))

os.listdir('../data/')

['abnormal']

In [25]:
path = '../data/org_train/'
n = int(len(txt_lst)*0.9)
if not(os.path.isdir(path)):
        os.makedirs(os.path.join(path))

# Train data
for data in txt_lst[:n]:
    org_data = '../src/' + data
    cop_data = path + data
    
    shutil.copyfile(org_data, cop_data)

print(len(os.listdir(path)))

for data in jpg_lst[:n]:
    org_data = '../src/' + data
    cop_data = path + data
    
    shutil.copyfile(org_data, cop_data)

print(len(os.listdir(path)))

2050
4100


In [26]:
path = '../data/org_test/'
n = int(len(txt_lst)*0.9)
if not(os.path.isdir(path)):
        os.makedirs(os.path.join(path))

# Test data
for data in txt_lst[n:]:
    org_data = '../src/' + data
    cop_data = path + data
    
    shutil.copyfile(org_data, cop_data)

print(len(os.listdir(path)))

for data in jpg_lst[n:]:
    org_data = '../src/' + data
    cop_data = path + data
    
    shutil.copyfile(org_data, cop_data)

print(len(os.listdir(path)))

228
456


# Preprocessing

이미지 전처리 및 annotation 파일 만들기

In [28]:
import torch
from torchvision import transforms
import albumentations.pytorch
import pandas as pd
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import albumentations as A

plt.rcParams["figure.figsize"]=10, 10

In [29]:
# bounding box 좌표 변경하기
def cvt_coordinate(center_x, center_y, w, h):
    '''
    start_x, start_y: 시작 꼭짓점 좌표 (x, y)
    end_x, end_y: 종료 꼭짓점 좌표 (x, y)
    '''
    start_x, end_x = center_x - (w/2), center_x + (w/2)
    start_y, end_y = center_y - (h/2), center_y + (h/2)

    return int(start_x), int(start_y), int(end_x), int(end_y)

## Train

In [31]:
# data path 설정
DATA_PATH = '../data/org_train/'
data_list = os.listdir(DATA_PATH)

print(len(data_list))

# 이미지와 라벨 데이터 
img_list = []
txt_list = []

for data in data_list:
    data = DATA_PATH + data
    if data.endswith('txt'):
        txt_list.append(data)
    else:
        img_list.append(data)

txt_list.sort()
img_list.sort()

print(len(img_list), len(txt_list))

4100
2050 2050


In [32]:
n = len(img_list)
target_size = 416

f = open('../data/train_annotation.txt', 'w')
for i in range(n):
#     print(img_list[i])
#     print(txt_list[i])
    
    image = cv2.imread(img_list[i])    
    df = pd.read_csv(txt_list[i])
    
    title = img_list[i].split('/')[-1]
    
    bboxes = []
    for i in range(6):
        label = str(df.iloc[i][0])
        sr = df.iloc[i][1:5].tolist()
        x1, y1, x2, y2 = cvt_coordinate(sr[0], sr[1], sr[2], sr[3])
        bboxes.append([x1, y1, x2, y2, label])
#     print('bboxes: ', bboxes)
    
    # Transform
    transform = A.Compose([
        A.LongestMaxSize(max_size=target_size, always_apply=True),
        A.PadIfNeeded(target_size, target_size, always_apply=True, border_mode=0, value=[0, 0, 0]),
        A.pytorch.transforms.ToTensor()
        ],
        bbox_params = A.BboxParams(format='pascal_voc')
    )
    data = transform(image=image, bboxes=bboxes)
        
    # show image with bbox and labels
    img = data['image']
    bboxes = data['bboxes']
    img = np.array(img.permute(1,2,0))
    img = (img*255).astype(np.uint8)
    
    cv2.imwrite('../data/train/' + title, img)

    # bounding box
    text = []
    for i in range(6):
        coor = bboxes[i]
        coor = list(map(int, coor))
#         img = cv2.rectangle(img, (coor[0], coor[1]), (coor[2], coor[3]), (0, 0, 255), 1)
#         cv2.putText(img, str(coor[4]), (coor[0], coor[1]), cv2.FONT_ITALIC, 0.5, (255, 0, 0), 1, cv2.LINE_AA)
        
        coor = list(map(str, coor))
        text.extend(coor)

    res = './data/train/'+ title + ' ' + ','.join(text) + '\n'
    f.write(res)

f.close()

## Test

In [33]:
# data path 설정
DATA_PATH = '../data/org_test/'
data_list = os.listdir(DATA_PATH)

print(len(data_list))

# 이미지와 라벨 데이터 
img_list = []
txt_list = []

for data in data_list:
    data = DATA_PATH + data
    if data.endswith('txt'):
        txt_list.append(data)
    else:
        img_list.append(data)

txt_list.sort()
img_list.sort()

print(len(img_list), len(txt_list))

456
228 228


In [34]:
n = len(img_list)
target_size = 416

f = open('../data/test_annotation.txt', 'w')
for i in range(n):
#     print(img_list[i])
#     print(txt_list[i])
    
    image = cv2.imread(img_list[i])    
    df = pd.read_csv(txt_list[i])
    
    title = img_list[i].split('/')[-1]
    
    bboxes = []
    for i in range(6):
        label = str(df.iloc[i][0])
        sr = df.iloc[i][1:5].tolist()
        x1, y1, x2, y2 = cvt_coordinate(sr[0], sr[1], sr[2], sr[3])
        bboxes.append([x1, y1, x2, y2, label])
#     print('bboxes: ', bboxes)
    
    # Transform
    transform = A.Compose([
        A.LongestMaxSize(max_size=target_size, always_apply=True),
        A.PadIfNeeded(target_size, target_size, always_apply=True, border_mode=0, value=[0, 0, 0]),
        A.pytorch.transforms.ToTensor()
        ],
        bbox_params = A.BboxParams(format='pascal_voc')
    )
    data = transform(image=image, bboxes=bboxes)
        
    # show image with bbox and labels
    img = data['image']
    bboxes = data['bboxes']
    img = np.array(img.permute(1,2,0))
    img = (img*255).astype(np.uint8)
    
    cv2.imwrite('../data/test/' + title, img)

    # bounding box
    text = []
    for i in range(6):
        coor = bboxes[i]
        coor = list(map(int, coor))
#         img = cv2.rectangle(img, (coor[0], coor[1]), (coor[2], coor[3]), (0, 0, 255), 1)
#         cv2.putText(img, str(coor[4]), (coor[0], coor[1]), cv2.FONT_ITALIC, 0.5, (255, 0, 0), 1, cv2.LINE_AA)
        
        coor = list(map(str, coor))
        text.extend(coor)

    res = './data/test/'+ title + ' ' + ','.join(text) + '\n'
    f.write(res)

f.close()

# Edit annotations txt

In [3]:
new_f = open('../data/train_annotation__.txt', mode='w')

with open('../data/train_annotation.txt', mode='r') as f:
    data = f.readlines()
    
    for i in range(len(data)):
        data_lst = data[i].split(',')
        title = data_lst[0]
#         title_ = title.replace('data__', 'data')        
        val = title.split(' ')
        data_lst = val + data_lst[1:]
        res = ''
        for j in range(len(data_lst)):
            res += data_lst[j]
            if j % 5 == 0:
                res += ' '
            else:
                res += ','
                    
        new_f.write(res[:-1])
    
new_f.close()

In [2]:
new_f = open('../data/test_annotation__.txt', mode='w')

with open('../data/test_annotation.txt', mode='r') as f:
    data = f.readlines()
    
    for i in range(len(data)):
        data_lst = data[i].split(',')
        title = data_lst[0]
#         title_ = title.replace('data__', 'data')        
        val = title.split(' ')
        data_lst = val + data_lst[1:]
        res = ''
        for j in range(len(data_lst)):
            res += data_lst[j]
            if j % 5 == 0:
                res += ' '
            else:
                res += ','
                    
        new_f.write(res[:-1])
    
new_f.close()